In [14]:
!pip install pymc3

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  Using cached pymc3-3.7.tar.gz (30.8 MB)
ERROR: Package 'pymc3' requires a different Python: 2.7.16 not in '>=3.5.4'


In [15]:
!pip install sklearn

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [3]:
import pymc3

import pandas as pd
import numpy as np

import os
import sys
import json
import datetime

from matplotlib import pyplot as plt
from scipy import stats, special

# sys.path.append('pyfail')

from pyfail import *
# import utils
# import links
# import generators
# import estimation
# import evaluation

# from sklearn import mixture

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
dir(pf)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [17]:
X = pd.read_csv('inputs/weather.csv', index_col='time')


In [18]:
# Specify mean and distribution of failure model parameters to run

In [19]:
beta = 0.2
alpha = 65
cov = 0.1

fleet_size = 100

In [20]:
# Generate failure model parameters from sampling distribution

In [21]:
names = ['alpha','beta']
params = generators.params([alpha, beta],
                           fleet_size, 
                           variance=[alpha*cov, None],
                           distributions=[stats.norm, None],
                           names=['alpha','beta'],
                          seed=0)

failures, failure_rate = generators.failure_data(params, X, links.Link, fleet_size=fleet_size)

df = pd.DataFrame(index=failures.index)
df['count'] = failures
df['rate'] = failure_rate



NameError: name 'links' is not defined

In [ ]:


# specify priors
alpha = pymc.Uniform('alpha', 0,100, value=np.average(50))
beta = pymc.Uniform('beta', 0.2, value=1)

# define link function relating failures to conditions X
@pymc.deterministic
def model(a=alpha, b=beta, x=X['Wind'], s=fleet_size):
    return s/(1+np.exp(-1*b*(x-a)))

failure_process = pymc.Poisson('failures', mu=model, value=failures, observed=True)


sampler = pymc.MCMC([alpha, beta, model, X['Wind'], failure_process, failures])

# sampler.use_step_method(pymc.AdaptiveMetropolis, [alpha, beta],
#                         scales={alpha:0.1, beta:0.1})

sampler.use_step_method(pymc.Gibbs, [alpha, beta], pymc.Normal('\hat{alpha}', ))

sampler.sample(iter=1000)